# **LOADING ALL 7 DATATES**

In [1]:
# Intializing Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Olist E-Commerce Analytics") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print(f"Spark {spark.version} initialized")
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/27 22:31:24 WARN Utils: Your hostname, Roberts-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.43 instead (on interface en0)
25/11/27 22:31:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/27 22:31:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark 4.0.1 initialized


In [2]:
# Loading Customers Dataset
customers = spark.read.csv("/Users/robert/Desktop/ecommerce-pipeline/data/raw/olist_customers_dataset.csv", header=True, inferSchema=True)

print(f"Customers: {customers.count():,} records")
print(f"Columns: {len(customers.columns)}")
customers.printSchema()
customers.show(5)

Customers: 99,441 records
Columns: 5
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruz

In [3]:
# Load Orders Dataset
orders = spark.read.csv("/Users/robert/Desktop/ecommerce-pipeline/data/raw/olist_orders_dataset.csv", header=True, inferSchema=True)

print(f"Orders: {orders.count():,} records")
print(f"Columns: {len(orders.columns)}")
orders.printSchema()
orders.show(5, truncate=False)

Orders: 99,441 records
Columns: 8
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+---------------------

In [4]:
# Load Order Items
order_items = spark.read.csv("/Users/robert/Desktop/ecommerce-pipeline/data/raw/olist_order_items_dataset.csv", header=True, inferSchema=True)

print(f"Order Items: {order_items.count():,} records")
order_items.printSchema()
order_items.show(5)

Order Items: 112,650 records
root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032e

In [5]:
# Loading Payments
payments = spark.read.csv("/Users/robert/Desktop/ecommerce-pipeline/data/raw/olist_order_payments_dataset.csv", header=True, inferSchema=True)

print(f"Payments: {payments.count():,} records")
payments.show(5)

Payments: 103,886 records
+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 5 rows


In [6]:
# Loading Reviews
reviews = spark.read.csv("/Users/robert/Desktop/ecommerce-pipeline/data/raw/olist_order_reviews_dataset.csv", header=True, inferSchema=True)

print(f"Reviews: {reviews.count():,} records")
reviews.show(5)

Reviews: 104,162 records
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                NULL|                  NULL| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                NULL|                  NULL| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                NULL|                  NULL| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                NULL|  Recebi bem antes ...

In [7]:
# Loading Products
products = spark.read.csv("/Users/robert/Desktop/ecommerce-pipeline/data/raw/olist_products_dataset.csv", header=True, inferSchema=True)

print(f"Products: {products.count():,} records")
products.show(5)

Products: 32,951 records
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|      

In [8]:
# Loading Sellers
sellers = spark.read.csv("/Users/robert/Desktop/ecommerce-pipeline/data/raw/olist_sellers_dataset.csv", header=True, inferSchema=True)

print(f"Sellers: {sellers.count():,} records")
sellers.show(5)

Sellers: 3,095 records
+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
+--------------------+----------------------+-----------------+------------+
only showing top 5 rows


In [9]:
# Summarizing the stats
print("\n" + "="*60)
print("DATASET SUMMARY")
print("="*60 + "\n")

datasets = {
    'Customers': customers,
    'Orders': orders,
    'Order Items': order_items,
    'Payments': payments,
    'Reviews': reviews,
    'Products': products,
    'Sellers': sellers
}

for name, df in datasets.items():
    print(f"{name:15} {df.count():>8,} records  |  {len(df.columns):>2} columns")

# Fixed: use Python's built-in sum, not Spark's sum
total_records = 0
for df in datasets.values():
    total_records += df.count()

print(f"\nTotal records: {total_records:,}")
print("="*60)


DATASET SUMMARY

Customers         99,441 records  |   5 columns
Orders            99,441 records  |   8 columns
Order Items      112,650 records  |   7 columns
Payments         103,886 records  |   5 columns
Reviews          104,162 records  |   7 columns
Products          32,951 records  |   9 columns
Sellers            3,095 records  |   4 columns

Total records: 555,626


In [10]:
# Quick Analysis for Order status
print("Order Status Distribution:")
orders.groupBy("order_status").count().orderBy(desc("count")).show()

Order Status Distribution:
+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [11]:
# Performing termporal analysis
print("Orders by Year-Month:")
orders_with_date = orders.withColumn("year_month", 
                                     date_format("order_purchase_timestamp", "yyyy-MM"))

orders_with_date.groupBy("year_month") \
    .count() \
    .orderBy("year_month") \
    .show(30)

Orders by Year-Month:
+----------+-----+
|year_month|count|
+----------+-----+
|   2016-09|    4|
|   2016-10|  324|
|   2016-12|    1|
|   2017-01|  800|
|   2017-02| 1780|
|   2017-03| 2682|
|   2017-04| 2404|
|   2017-05| 3700|
|   2017-06| 3245|
|   2017-07| 4026|
|   2017-08| 4331|
|   2017-09| 4285|
|   2017-10| 4631|
|   2017-11| 7544|
|   2017-12| 5673|
|   2018-01| 7269|
|   2018-02| 6728|
|   2018-03| 7211|
|   2018-04| 6939|
|   2018-05| 6873|
|   2018-06| 6167|
|   2018-07| 6292|
|   2018-08| 6512|
|   2018-09|   16|
|   2018-10|    4|
+----------+-----+



In [12]:
# Performing payment types analysis
print("Payment Types Distribution:")
payments.groupBy("payment_type").count().orderBy(desc("count")).show()

Payment Types Distribution:
+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [13]:
# The average order value
print("Average Order Metrics:")
order_items.select(
    avg("price").alias("avg_price"),
    min("price").alias("min_price"),
    max("price").alias("max_price"),
    avg("freight_value").alias("avg_freight")
).show()

Average Order Metrics:
+------------------+---------+---------+------------------+
|         avg_price|min_price|max_price|       avg_freight|
+------------------+---------+---------+------------------+
|120.65373901464174|     0.85|   6735.0|19.990319928982977|
+------------------+---------+---------+------------------+



25/11/28 03:31:46 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1984496 ms exceeds timeout 120000 ms
25/11/28 03:31:46 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/28 03:31:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$